# Solving the Phillips-Wunsch-Garrett IVP with non-constant coefficients using Dedalus

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py
from dedalus import public as de
from dedalus.extras import flow_tools
from dedalus.extras import plot_tools
from dedalus.tools import post
import pathlib
import time
from IPython import display

import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")

logger = logging.getLogger(__name__)

# Problem formulation

### Physical parameters

In [2]:
N = 1.3e-3
f = 0.53e-4

# topographic parameters
slopeAngle = 2.e-3
tht = slopeAngle

# mixing parameters
d = 230
k0 = 5.2e-5
k1 = 1.8e-3
Pr = 1

In [3]:
def calc_delta():
    return ((4*Pr**2*(k0+k1)**2) / (f**2*np.cos(tht)**2 * (1 + (N**2*np.tan(tht)**2/(f**2))*Pr)))**0.25

In [40]:
#===== Set up domain =====
# Create basis and domain for all experiments
Lz = 4000
nz = 512
z_basis = de.Chebyshev('z', nz, interval=(0, Lz), dealias=3/2)
domain = de.Domain([z_basis], np.float64)
z = domain.grid(0)

#===== Set up Problem ======
problem = de.IVP(domain, variables=['u', 'v', 'b', 'uz', 'vz', 'bz']);

# Set parameters
problem.parameters['f'] = f
problem.parameters['tht'] = tht # set in loop
problem.parameters['N'] = N
k = domain.new_field(name='k')
k['g'] = k0+k1*np.exp(-z/d)
problem.parameters['k'] = k

nu = domain.new_field(name='nu')
nu['g'] = (k0+k1*np.exp(-z/d))*Pr
problem.parameters['nu'] = nu

# Main equations
problem.add_equation("-f*cos(tht)*v - sin(tht)*b - dz(nu*uz) = 0")
problem.add_equation("f*u*cos(tht) - dz(nu*vz) = 0")
problem.add_equation("dt(b) + N**2*sin(tht)*u - dz(k*bz) = N**2*cos(tht)*dz(k)")

# Auxiliary equations defining the first-order reduction
problem.add_equation("uz - dz(u) = 0")
problem.add_equation("vz - dz(v) = 0")
problem.add_equation("bz - dz(b) = 0")

# Boundary conditions
problem.add_bc('left(u) = 0')
problem.add_bc('left(v) = 0')
problem.add_bc('left(bz) = - N**2*cos(tht)')
problem.add_bc('right(uz) = 0')
problem.add_bc('right(vz) = 0')
problem.add_bc('right(bz) = 0')

# Set solver for IVP
solver = problem.build_solver(de.timesteppers.RK443);

#==== Set initial conditions ====
# Reference local grid and state fields
z = domain.grid(0)
u = solver.state['u']
v = solver.state['v']
b = solver.state['b']
uz = solver.state['uz']
vz = solver.state['vz']
bz = solver.state['bz']

# State from a state of rest
u['g'] = np.zeros_like(z)
v['g'] = np.zeros_like(z)
b['g'] = np.zeros_like(z)
u.differentiate('z', out=uz)
v.differentiate('z', out=vz)
b.differentiate('z', out=bz)


#==== Create analysis files ====
output_path = '../../data/dedalus/basic_state'
analysis = solver.evaluator.add_file_handler(output_path, iter=1)
analysis.add_system(solver.state, layout='g')

# Stop stopping criteria
solver.stop_sim_time = (50.*24.*60.*60.)
solver.stop_wall_time = np.inf
solver.stop_iteration = np.inf

# Setup storage
u_list = [np.copy(u['g'])]
v_list = [np.copy(v['g'])]
b_list = [np.copy(b['g'])]
bz_list = [np.copy(bz['g'])]
t_list = [solver.sim_time]

# Main loop
dt = 3*60.*60.
start_time = time.time()
print('Iterations: ')
while solver.ok:
    solver.step(dt);
    if solver.sim_time % (3.*60.*60.) == 0:
        print(np.int64(solver.sim_time/60.), end=", ")
        u_list.append(np.copy(u['g']))
        v_list.append(np.copy(v['g']))
        b_list.append(np.copy(b['g']))
        bz_list.append(np.copy(bz['g']))
        t_list.append(solver.sim_time)
        
end_time = time.time()
print('Runtime:', end_time-start_time)
z_da = domain.grid(0, scales=domain.dealias)

2020-01-13 15:07:50,271 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 9.3e+00/s
Iterations: 


/home/hdrake/.conda/envs/bblturb/lib/python3.7/site-packages/dedalus/core/evaluator.py:533: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  dset.dims.create_scale(scale, sn)
/home/hdrake/.conda/envs/bblturb/lib/python3.7/site-packages/dedalus/core/evaluator.py:550: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  dset.dims.create_scale(scale, lookup)


180, 360, 540, 720, 900, 1080, 1260, 1440, 1620, 1800, 1980, 2160, 2340, 2520, 2700, 2880, 3060, 3240, 3420, 3600, 3780, 3960, 4140, 4320, 4500, 4680, 4860, 5040, 5220, 5400, 5580, 5760, 5940, 6120, 6300, 6480, 6660, 6840, 7020, 7200, 7380, 7560, 7740, 7920, 8100, 8280, 8460, 8640, 8820, 9000, 9180, 9360, 9540, 9720, 9900, 10080, 10260, 10440, 10620, 10800, 10980, 11160, 11340, 11520, 11700, 11880, 12060, 12240, 12420, 12600, 12780, 12960, 13140, 13320, 13500, 13680, 13860, 14040, 14220, 14400, 14580, 14760, 14940, 15120, 15300, 15480, 15660, 15840, 16020, 16200, 16380, 16560, 16740, 16920, 17100, 17280, 17460, 17640, 17820, 18000, 18180, 18360, 18540, 18720, 18900, 19080, 19260, 19440, 19620, 19800, 19980, 20160, 20340, 20520, 20700, 20880, 21060, 21240, 21420, 21600, 21780, 21960, 22140, 22320, 22500, 22680, 22860, 23040, 23220, 23400, 23580, 23760, 23940, 24120, 24300, 24480, 24660, 24840, 25020, 25200, 25380, 25560, 25740, 25920, 26100, 26280, 26460, 26640, 26820, 27000, 27180, 273

In [ ]:
from dedalus.tools import post
import pathlib
post.merge_process_files(output_path, cleanup=True)
set_paths = list(pathlib.Path(output_path).glob("basic_state_s*.h5"))
os.system(f"rm {output_path}/output.h5")
post.merge_sets(output_path+"/output.h5", set_paths, cleanup=True)